In [1]:
import os
import numpy as np
import pandas as pd
import json
from copy import deepcopy
import re
# -*- coding:utf-8 -*- 

# QA训练集

In [2]:
# 证据等级
col_name = "证据等级"
col_ab = "evi"
que_d = {"1":"Where is pathogen isolated?"}

# 免疫状态
col_name = "免疫状态"
col_ab = "immu"
que_d = {"1":"What's patients immunity state?"}

# 人数
col_name = "样本量"
col_ab = "number"
que_d = {"1":"How many patients are ivolved in this study?"}

# 性别
col_name = "性别"
col_ab = "gender"
que_d = {"1":"What's gender of the patient?"}
# 年龄
col_name = "年龄"
col_ab = "age"
que_d = {"1":"How old is patient?"}

In [3]:
sped_path = "../5.Orgnize/文献标注-校对/"
sped_l = [ele for ele in os.listdir(sped_path) if 'more' in ele]
txt_path = "../4.Label_prepare/paper_ld_old/"
train_qa = []

for sped in sped_l:

    # 读取标注信息
    print(sped)
    df = pd.read_excel("%s/%s" % (sped_path, sped), header=0)
    df = df.loc[df["是否是致病菌"] == "是",:]

    # 读取标题和摘要的列用于后续添加标题和摘要
    df["Full"] = 0

    for idx in df.index.tolist():

        with open("%s/%s/%s.txt" % (txt_path, sped.split("_")[0], df.loc[idx, "ID"])) as f:
            txt = f.read()
        df.loc[idx, "Full"] = txt

        # 创建WHAT Question样本
        if np.isnan(df.loc[idx,"%s_old_start" % col_ab]) == False:
            if type(df.loc[idx,"%s_answer" % col_ab]) == float:
                df.loc[idx,"%s_answer" % col_ab] = "NaN"
            if df.loc[idx, "%s_old_start" % col_ab] < len(df.loc[idx, "Full"]):
                qa = {
                    "qas": [
                        {
                            "id": str(df.loc[idx, "ID"]),
                            "question": que_d["1"],
                            "answers": [
                                {
                                    "text": df.loc[idx, "%s_answer" % col_ab],
                                    "answer_start": int(df.loc[idx, "%s_old_start" % col_ab]),
                                }
                            ]
                        }
                    ],
                    "context": df.loc[idx, "Full"]}
                train_qa.append(qa)
    
    df.to_csv("input/input_%s/%s.csv" % (col_ab, sped.split("_")[0]), header=True, index=False)

train_set = {
    "data": [{
        'paragraphs': train_qa,
        'title': "LungB"
    }],
    "version": "LungB"
}
json.dump(train_set, open("input/input_%s/train_set.json" % (col_ab), "w"))


In [ ]:

for sped in sped_l:

    # 读取标注信息
    print(sped)
    df = pd.read_excel("%s/%s" % (sped_path, sped), header=0)
    df = df.loc[df["是否是致病菌"] == "是",:]

    # 读取标题和摘要的列用于后续添加标题和摘要
    df["Full"] = 0

    for idx in df.index.tolist():

        with open("%s/%s/%s.txt" % (txt_path, sped.split("_")[0], df.loc[idx, "ID"])) as f:
            txt = f.read()
        df.loc[idx, "Full"] = txt

        # 创建WHAT Question样本
        if np.isnan(df.loc[idx,"%s_old_start" % col_ab]) == False:
            if type(df.loc[idx,"%s_answer" % col_ab]) == float:
                df.loc[idx,"%s_answer" % col_ab] = "NaN"
            if df.loc[idx, "%s_old_start" % col_ab] < len(df.loc[idx, "Full"]):
                qa = {
                    "qas": [
                        {
                            "id": str(df.loc[idx, "ID"]),
                            "question": que_d["1"],
                            "answers": [
                                {
                                    "text": df.loc[idx, "%s_answer" % col_ab],
                                    "answer_start": int(df.loc[idx, "%s_old_start" % col_ab]),
                                }
                            ]
                        }
                    ],
                    "context": df.loc[idx, "Full"]}
                train_qa.append(qa)
    
    df.to_csv("input/input_%s/%s.csv" % (col_ab, sped.split("_")[0]), header=True, index=False)

train_set = {
    "data": [{
        'paragraphs': train_qa,
        'title': "LungB"
    }],
    "version": "LungB"
}
json.dump(train_set, open("input/input_%s/train_set.json" % (col_ab), "w"))


# 预测

In [9]:
prediction = json.load(open("output/predictions_.json", 'r'))
result = {}
for i in prediction.keys():
    toke = i.split(":")[0]
    species = ""
    for j in toke.split("_")[0:-1]:
        species = species + " " + j
    ncbi = "https://pubmed.ncbi.nlm.nih.gov/%s/" % toke.split("_")[-1]

    result[toke] = [toke.split("_")[-1], species, prediction[i][1][0], ncbi]
    
result = pd.DataFrame(result, index = ["ID", "SPECIES", "rate", "link"]).T

In [14]:
sped_path = "../4.Label_prepare/paper_md"
sped_l = [ele for ele in os.listdir(sped_path) if '.' not in ele]

test_qa = []

for sped in sped_l:
    
    df = pd.DataFrame(np.zeros((0,2)), columns = ["SPECIES", "Full"])
    
    id_l = [ele for ele in os.listdir("%s/%s" % (sped_path, sped)) if 'txt' in ele]
    id_l = [ele for ele in id_l if ele.replace(".txt", "") in result.index]

    for idx in id_l:

        if result.loc[idx.replace(".txt", ""), "rate"] > 0.5:
        
            with open("%s/%s/%s" % (sped_path, sped, idx)) as f:
                txt0 = f.readlines()
            with open("%s/%s/%s" % (sped_path, sped, idx)) as f:
                txt = f.read()
            
            idx = idx.split(".")[0]
            df.loc[idx] = [txt0[0].split(":")[1], txt]
            
            qa = {'qas': [{
                'id': '%s:%s' % (idx, df.loc[idx, "SPECIES"]),
                'question': que_d[0],
                "answers": []}],
                'context': '%s' % df.loc[idx, "Full"]}
            test_qa.append(qa)
        
    df.to_csv("input/input_%s/test/%s_immu.csv" % (col_ab, sped), header=True, index=True)

test_set = {"data":[{'paragraphs':test_qa, 'title':"LungB"}], "version":"LungB"}

json.dump(test_set, open("input/input_%s/test_set.json" % col_ab, "w"))

Anaerobic
Curved
GramN
GramPc
GramPr
Intracellular
